# FEM Modelling for STEDIN Transformer

In this part, the conducitvity of the core material is modelled as $2.22 \times 10^{6}$ (grain-oriented Fe-Si steel) [1].

[1] Fiorillo, Fausto. Characterization and measurement of magnetic materials. Academic Press, 2004.

## Import Packages

In [1]:
using gmsh
using LinearAlgebra
using LaTeXStrings
include("../../../lib/Makie_Plots.jl");
using .Makie_Plots
include("../../../lib/FEM_Tri_1e.jl");
using .FEM_Tri_1e
include("../../../lib/Mesh_Data_stedin.jl");
using .Mesh_Data_stedin
include("../../../lib/Save_VTK.jl");
using .Save_VTK

## Julia code

In [2]:
gmsh.initialize()

## Read mesh from file
gmsh.open("../mesh/stedin_transformer.msh")

mesh_data = get_mesh_data_tri_1e(gmsh)

Ip = 0;       # Primary peak phase current
Is = 777.62;  # Secondary peak phase current
Np = 266;
Ns = 6;

omega = 2*pi*50;  # Frequency

# HV winding dimensions (all phases left/right are identical)
wwhv = 3e-2;
hwhv = 74e-2;
mwhv = 14.75e-2;
Awhv = wwhv * hwhv;

# LV winding dimensions (all phases left/right are identical)
wwlv = 2e-2;
hwlv = 74e-2;
mwlv = 11.25e-2;
Awlv = wwlv * hwlv;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Source current density J
# One term for each of the windings, with a positive and negative part
# Note the phase shift between the phases
sourcefunction(group_id) = Jp * exp(1im * 2pi/3) * (-1 * (group_id==3) + 1 * (group_id==4)) + 
                           Jp * (-1 * (group_id==5) + 1 * (group_id==6)) + 
                           Jp * exp(-1im * 2pi/3) * (-1 * (group_id==7) + 1 * (group_id==8)) + 
                           Js * exp(1im * 2pi/3) * (1 * (group_id==9) - 1 * (group_id==10)) +
                           Js * (1 * (group_id==11) - 1 * (group_id==12)) + 
                           Js * exp(-1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14));
sourceperelement = map(sourcefunction, mesh_data.e_group);

# Relative permeability model
mu0 = 4e-7 * pi;
mur = 1000;       # Relative permeability of the core
reluctivityfunction(group_id) = (1 / mu0) + (1/(mu0*mur) - 1/mu0) * (group_id == 2);
reluctivityperelement = map(reluctivityfunction, mesh_data.e_group);

# Conductivity
conductivityfunction(group_id) = 1e-10 + (2.222e6 - 1e-10) * (group_id == 2);
conductivityperelement = map(conductivityfunction, mesh_data.e_group);

# Handle the boundary conditions
bnd_node_ids, _ = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1);

# Calculate the vector potential
u = fem(mesh_data, sourceperelement, reluctivityperelement, conductivityperelement, omega, bnd_node_ids);

# Post-process for magnetic field and current density
Bx, By, B, Hx, Hy, H, mag_energy, Jel = post_process(mesh_data, u, sourceperelement, reluctivityperelement, conductivityperelement, omega);

# Contour plot of the magnetic flux density
plot_surface_tri_1e(norm.(B), "stedin_transformer_B_plot_with_eddy", gmsh)
# Contour plot of the magnetic field strength
plot_surface_tri_1e(norm.(H), "stedin_transformer_H_plot_with_eddy", gmsh)
# Contour plot of the magnetic energy
plot_surface_tri_1e(norm.(mag_energy), "stedin_transformer_mag_energy_plot_with_eddy", gmsh)

save_vtk(mesh_data, norm.(u), norm.(B), norm.(H), norm.(mag_energy), norm.(Jel), reluctivityperelement, "stedin_transformer_mag_energy_plot_with_eddy")

gmsh.finalize()

Info    : Reading '../mesh/stedin_transformer.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading '../mesh/stedin_transformer.msh'


### STEDIN Transformer : Magnetic field density
<img src="../img/stedin_transformer_B_plot_with_eddy.png">

### STEDIN Transformer : Magnetic field strength
<img src="../img/stedin_transformer_H_plot_with_eddy.png">

### STEDIN Transformer : Magnetic energy
<img src="../img/stedin_transformer_mag_energy_plot_with_eddy.png">